
Proximal Policy Optimization
==========


PPO is motivated by the same question as TRPO: how can we take the biggest possible improvement step on a policy using the data we currently have, without stepping so far that we accidentally cause performance collapse? Where TRPO tries to solve this problem with a complex second-order method, <span style="color:red">PPO is a family of first-order methods that use a few other tricks to keep new policies close to old.</span> PPO methods are significantly simpler to implement, and empirically seem to perform at least as well as TRPO.

There are two primary variants of PPO: PPO-Penalty and PPO-Clip. 

**PPO-Penalty** approximately solves a KL-constrained update like TRPO, but penalizes the KL-divergence in the objective function instead of making it a hard constraint, and automatically adjusts the penalty coefficient over the course of training so that it's scaled appropriately. 

**<span style="color:red">PPO-Clip</span>** doesn't have a KL-divergence term in the objective and doesn't have a constraint at all. Instead relies on specialized clipping in the objective function to remove incentives for the new policy to get far from the old policy. 

Here, we'll focus only on PPO-Clip (the primary variant used at OpenAI).

Quick Facts
-----------

* PPO is an on-policy algorithm.
* PPO can be used for environments with either discrete or continuous action spaces.
* The Spinning Up implementation of PPO supports parallelization with MPI.

Key Equations
-------------

PPO-clip updates policies via

\begin{align*}
    \theta_{k+1} = \arg \max_{\theta} \underset{s,a \sim \pi_{\theta_k}}{{\mathrm E}}\left[
        L(s,a,\theta_k, \theta)\right],
\end{align*}

typically taking multiple steps of (usually minibatch) SGD to maximize the objective. Here $L$ is given by

\begin{align*}
    L(s,a,\theta_k,\theta) = \min\left(
    \frac{\pi_{\theta}(a|s)}{\pi_{\theta_k}(a|s)}  A^{\pi_{\theta_k}}(s,a), \;\;
    \text{clip}\left(\frac{\pi_{\theta}(a|s)}{\pi_{\theta_k}(a|s)}, 1 - \epsilon, 1+\epsilon \right) A^{\pi_{\theta_k}}(s,a)
    \right),
\end{align*}

in which $\epsilon$ is a (small) hyperparameter which roughly says how far away the new policy is allowed to go from the old.

This is a pretty complex expression, and it's hard to tell at first glance what it's doing, or how it helps keep the new policy close to the old policy. As it turns out, there's a considerably simplified version [1]_ of this objective which is a bit easier to grapple with (and is also the version we implement in our code):

\begin{align*}
    L(s,a,\theta_k,\theta) = \min\left(
    \frac{\pi_{\theta}(a|s)}{\pi_{\theta_k}(a|s)}  A^{\pi_{\theta_k}}(s,a), \;\;
    g(\epsilon, A^{\pi_{\theta_k}}(s,a))
    \right),
\end{align*}

where

\begin{align*}
    g(\epsilon, A) = \left\{ 
        \begin{array}{ll}
        (1 + \epsilon) A & A \geq 0 \\
        (1 - \epsilon) A & A < 0.
        \end{array}
        \right.
\end{align*}

To figure out what intuition to take away from this, let's look at a single state-action pair $(s,a)$, and think of cases. 

**Advantage is positive**: Suppose the advantage for that state-action pair is positive, in which case its contribution to the objective reduces to

\begin{align*}
    L(s,a,\theta_k,\theta) = \min\left(
    \frac{\pi_{\theta}(a|s)}{\pi_{\theta_k}(a|s)}, (1 + \epsilon)
    \right)  A^{\pi_{\theta_k}}(s,a).
\end{align*}

Because the advantage is positive, the objective will increase if the action becomes more likely---that is, if $\pi_{\theta}(a|s)$ increases. But the min in this term puts a limit to how *much* the objective can increase. Once $\pi_{\theta}(a|s) > (1+\epsilon) \pi_{\theta_k}(a|s)$, the min kicks in and this term hits a ceiling of $(1+\epsilon) A^{\pi_{\theta_k}}(s,a)$. Thus: *the new policy does not benefit by going far away from the old policy*.

**Advantage is negative**: Suppose the advantage for that state-action pair is negative, in which case its contribution to the objective reduces to

\begin{align*}
    L(s,a,\theta_k,\theta) = \max\left(
    \frac{\pi_{\theta}(a|s)}{\pi_{\theta_k}(a|s)}, (1 - \epsilon)
    \right)  A^{\pi_{\theta_k}}(s,a).
\end{align*}

Because the advantage is negative, the objective will increase if the action becomes less likely---that is, if $\pi_{\theta}(a|s)$ decreases. But the max in this term puts a limit to how *much* the objective can increase. Once $\pi_{\theta}(a|s) < (1-\epsilon) \pi_{\theta_k}(a|s)$, the max kicks in and this term hits a ceiling of $(1-\epsilon) A^{\pi_{\theta_k}}(s,a)$. Thus, again: *the new policy does not benefit by going far away from the old policy*.

What we have seen so far is that<font color=red>clipping serves as a regularizer by removing incentives for the policy to change dramatically, and the hyperparameter $\epsilon$ corresponds to how far away the new policy can go from the old while still profiting the objective </font>.

**.. admonition:: You Should Know**

<font color= red>**[Problem]** While this kind of clipping goes a long way towards ensuring reasonable policy updates, it is still possible to end up with a new policy which is too far from the old policy</font>, and there are a bunch of tricks used by different PPO implementations to stave this off. In our implementation here, we use a particularly simple method: early stopping. If the mean KL-divergence of the new policy from the old grows beyond a threshold, we stop taking gradient steps. 
When you feel comfortable with the basic math and implementation details, it's worth checking out other implementations to see how they handle this issue!

Exploration vs. Exploitation
----------------------------

PPO trains a stochastic policy in an on-policy way. This means that it explores by sampling actions according to the latest version of its stochastic policy. The amount of randomness in action selection depends on both initial conditions and the training procedure. Over the course of training, the policy typically becomes progressively less random, as the update rule encourages it to exploit rewards that it has already found. This may cause the policy to get trapped in local optima.

# Pseudocode
![title](https://spinningup.openai.com/en/latest/_images/math/e62a8971472597f4b014c2da064f636ffe365ba3.svg)